# Sentinel-5P Monthly Comparisons

## Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Imports
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gpd
import rasterio as rio
import earthpy as et
import earthpy.plot as ep
import radiance as rd
import sentinel as stl

In [ ]:
# Set Options
sns.set(font_scale=1.5, style="whitegrid")

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

## Data Acquisition and Preprocessing

### Raster

In [ ]:
# Get all mean files into list
monthly_means = glob.glob(
    os.path.join(
        "03-processed-data",
        "raster",
        "south-korea",
        "statistics",
        "monthly",
        "*MEAN*tif",
    )
)

In [ ]:
# Read monthly mean GeoTiffs into arrays and store in dictionary
no2_dict = {}
for geotiff in monthly_means:
    # Read GeoTiff into array
    with rio.open(geotiff) as src:
        array = src.read(1)
    # Replace NAN values with 0.0 if necessary
    # Counts number of isnan() results that are True
    if np.count_nonzero(np.isnan(array)) > 0:
        np.nan_to_num(array, copy=False, nan=0.0)
    # CONVERT UNITS FROM MOL/M2 to MOLECULES/CM2
    # Store in dictionary, indexed by dictionary[YEAR][MONTH]
    year, month = (
        os.path.basename(geotiff)[16:20],
        os.path.basename(geotiff)[20:22],
    )
    if year not in no2_dict.keys():
        no2_dict[year] = {}
    if month not in no2_dict[year].keys():
        no2_dict[year][month] = array

In [ ]:
# Check top-level keys (years)
no2_dict.keys()

In [ ]:
# Check second-level keys (months)
no2_dict["2020"].keys()

In [ ]:
# Check single entry
no2_dict["2020"]["05"].shape

### Vector

In [ ]:
# Extract plotting extent
south_korea_extent = stl.extract_plotting_extent(monthly_means[0])
print(f"South Korea Extent: {south_korea_extent}")

In [ ]:
# Define path to South Korea shapefile
south_korea_shp_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea.shp"
)

# Read South Korea shapefile into geodataframe and project if necessary
south_korea_boundary = stl.project_vector(
    vector_path=south_korea_shp_path, raster_path=monthly_means[0]
)

# Create extent polygon
south_korea_polygon = stl.create_polygon_from_extent(south_korea_extent)

print(f"South Korea CRS: {south_korea_boundary.crs}")
print(f"South Korea Bounds:\n{south_korea_boundary.bounds.loc[0]}")

In [ ]:
# Define path to North Korea shapefile
north_korea_shp_path = os.path.join(
    "02-raw-data", "vector", "north-korea", "gadm36_north_korea.shp"
)

# Read North Korea shapefile into geodataframe and project if necessary
north_korea_boundary = stl.project_vector(
    vector_path=north_korea_shp_path, raster_path=monthly_means[0]
)

print(f"North Korea CRS: {north_korea_boundary.crs}")
print(f"North Korea Bounds:\n{north_korea_boundary.bounds.loc[0]}")

# Clip North Korea to South Korea extent polygon
north_korea_boundary_clipped = gpd.clip(
    gpd.GeoDataFrame(
        geometry=[south_korea_polygon], crs=north_korea_boundary.crs
    ),
    north_korea_boundary,
)

print(f"\nNorth Korea Clipped CRS: {north_korea_boundary_clipped.crs}")
print(
    f"North Korea Clipped Bounds:\n{north_korea_boundary_clipped.bounds.loc[0]}"
)

In [ ]:
# Display clipped boundary
north_korea_boundary_clipped.boundary.plot()

## Data Processing

### Magnitude Change

In [ ]:
# Calculate magnitude change
magnitude_change_jul_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["07"], post_change=no2_dict["2019"]["07"]
)

magnitude_change_aug_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["08"], post_change=no2_dict["2019"]["08"]
)

magnitude_change_sep_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["09"], post_change=no2_dict["2019"]["09"]
)

magnitude_change_oct_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["10"], post_change=no2_dict["2019"]["10"]
)

magnitude_change_nov_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["11"], post_change=no2_dict["2019"]["11"]
)

magnitude_change_dec_2018_2019 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2018"]["12"], post_change=no2_dict["2019"]["12"]
)

magnitude_change_jan_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["01"], post_change=no2_dict["2020"]["01"]
)

magnitude_change_feb_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["02"], post_change=no2_dict["2020"]["02"]
)

magnitude_change_mar_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["03"], post_change=no2_dict["2020"]["03"]
)

magnitude_change_apr_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["04"], post_change=no2_dict["2020"]["04"]
)

magnitude_change_may_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["05"], post_change=no2_dict["2020"]["05"]
)

magnitude_change_jun_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["06"], post_change=no2_dict["2020"]["06"]
)

magnitude_change_jul_2019_2020 = stl.calculate_magnitude_change(
    pre_change=no2_dict["2019"]["07"], post_change=no2_dict["2020"]["07"]
)

### Percent Change

In [ ]:
# Calculate percent change
percent_change_jul_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["07"], post_change=no2_dict["2019"]["07"]
)

percent_change_aug_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["08"], post_change=no2_dict["2019"]["08"]
)

percent_change_sep_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["09"], post_change=no2_dict["2019"]["09"]
)

percent_change_oct_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["10"], post_change=no2_dict["2019"]["10"]
)

percent_change_nov_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["11"], post_change=no2_dict["2019"]["11"]
)

percent_change_dec_2018_2019 = stl.calculate_percent_change(
    pre_change=no2_dict["2018"]["12"], post_change=no2_dict["2019"]["12"]
)

percent_change_jan_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["01"], post_change=no2_dict["2020"]["01"]
)

percent_change_feb_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["02"], post_change=no2_dict["2020"]["02"]
)

percent_change_mar_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["03"], post_change=no2_dict["2020"]["03"]
)

percent_change_apr_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["04"], post_change=no2_dict["2020"]["04"]
)

percent_change_may_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["05"], post_change=no2_dict["2020"]["05"]
)

percent_change_jun_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["06"], post_change=no2_dict["2020"]["06"]
)

percent_change_jul_2019_2020 = stl.calculate_percent_change(
    pre_change=no2_dict["2019"]["07"], post_change=no2_dict["2020"]["07"]
)

## Data Visualization

### Monthly Means

In [ ]:
# Create stack of all months (25 in total)
no2_months_stack = np.stack(
    [
        no2_dict[year][month]
        for year in no2_dict.keys()
        for month in no2_dict[year].keys()
    ]
)

# Check shape (bands, rows, columns)
print(f"Data stack shape (bands, rows, columns): {no2_months_stack.shape}")

# Check if the order is correct; compare first month of data (July 2018)
print(
    f"Data order correct: {np.array_equal(no2_months_stack[0], no2_dict['2018']['07'])}"
)

In [ ]:
# Set titles for data stack
titles = [
    "Jul 2018",
    "Aug 2018",
    "Sep 2018",
    "Oct 2018",
    "Nov 2018",
    "Dec 2018",
    "Jan 2019",
    "Feb 2019",
    "Mar 2019",
    "Apr 2019",
    "May 2019",
    "Jun 2019",
    "Jul 2019",
    "Aug 2019",
    "Sep 2019",
    "Oct 2019",
    "Nov 2019",
    "Dec 2019",
    "Jan 2020",
    "Feb 2020",
    "Mar 2020",
    "Apr 2020",
    "May 2020",
    "Jun 2020",
    "Jul 2020",
]

# Plot all stack bands (monthly means)
with plt.style.context("dark_background"):
    # Initialize figure/axes
    fig, ax = plt.subplots(nrows=8, ncols=3, figsize=(20, 40))
    plt.suptitle("South Korea NO2 Monthly Means, Jul 2018 - Jun 2020", size=24)
    plt.subplots_adjust(top=0.955)

    # Loop through all axes
    for (band, axis) in enumerate(fig.axes):
        # Plot monthly mean NO2 data
        ep.plot_bands(
            arr=no2_months_stack[band],
            cmap="inferno",
            title=titles[band],
            extent=south_korea_extent,
            vmin=0,
            vmax=no2_months_stack.max(),
            ax=axis,
        )

        # Add country boundaries and legend to axes
        north_korea_boundary_clipped.boundary.plot(
            edgecolor="#e41a1c",
            linewidth=0.5,
            ax=axis,
            alpha=1,
            label="North Korea",
        )
        south_korea_boundary.boundary.plot(
            edgecolor="#1b7837",
            linewidth=0.5,
            ax=axis,
            alpha=1,
            label="South Korea",
        )
        axis.legend(
            shadow=True, edgecolor="white", fontsize=10, loc="lower right"
        )

    # Add caption
    fig.text(
        0.5,
        0.11,
        f"Data Source: European Space Agency",
        ha="center",
        fontsize=16,
    )

### Monthly Comparison

#### Magnitude Change

In [ ]:
# Plot July 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["07"],
    post_change=no2_dict["2019"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "July 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "July 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot August 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["08"],
    post_change=no2_dict["2019"]["08"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "August 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "August 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot September 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["09"],
    post_change=no2_dict["2019"]["09"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "September 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "September 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot October 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["10"],
    post_change=no2_dict["2019"]["10"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "October 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "October 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot November 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["11"],
    post_change=no2_dict["2019"]["11"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "November 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "November 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot December 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["12"],
    post_change=no2_dict["2019"]["12"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "December 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "December 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot January 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["01"],
    post_change=no2_dict["2020"]["01"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "January 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "January 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot February 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["02"],
    post_change=no2_dict["2020"]["02"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "February 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "February 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot March 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["03"],
    post_change=no2_dict["2020"]["03"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "March 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "March 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot April 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["04"],
    post_change=no2_dict["2020"]["04"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "April 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "April 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot May 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["05"],
    post_change=no2_dict["2020"]["05"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "May 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "May 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot June 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["06"],
    post_change=no2_dict["2020"]["06"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "June 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "June 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot July 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["07"],
    post_change=no2_dict["2020"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "Jul 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Jul 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

#### Percent Change

In [ ]:
# Plot July 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["07"],
    post_change=no2_dict["2019"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "July 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "July 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot August 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["08"],
    post_change=no2_dict["2019"]["08"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "August 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "August 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot September 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["09"],
    post_change=no2_dict["2019"]["09"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "September 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "September 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot October 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["10"],
    post_change=no2_dict["2019"]["10"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "October 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "October 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot November 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["11"],
    post_change=no2_dict["2019"]["11"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "November 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "November 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot December 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["12"],
    post_change=no2_dict["2019"]["12"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "December 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "December 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot January 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["01"],
    post_change=no2_dict["2020"]["01"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "January 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "January 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot February 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["02"],
    post_change=no2_dict["2020"]["02"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "February 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "February 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot March 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["03"],
    post_change=no2_dict["2020"]["03"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "March 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "March 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot April 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["04"],
    post_change=no2_dict["2020"]["04"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "April 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "April 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot May 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["05"],
    post_change=no2_dict["2020"]["05"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "May 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "May 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot June 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["06"],
    post_change=no2_dict["2020"]["06"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "June 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "June 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

In [ ]:
# Plot July 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["07"],
    post_change=no2_dict["2020"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "Jul 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Jul 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

# Data Export

### Monthly Means

In [ ]:
# Plot and export all stack bands (monthly means)
with plt.style.context("dark_background"):
    # Initialize figure/axes
    fig, ax = plt.subplots(nrows=8, ncols=3, figsize=(20, 40))
    plt.suptitle("South Korea NO2 Monthly Means, Jul 2018 - Jun 2020", size=24)
    plt.subplots_adjust(top=0.955)

    # Loop through all axes
    for (band, axis) in enumerate(fig.axes):
        # Plot monthly mean NO2 data
        ep.plot_bands(
            arr=no2_months_stack[band],
            cmap="inferno",
            title=titles[band],
            extent=south_korea_extent,
            vmin=0,
            vmax=no2_months_stack.max(),
            ax=axis,
        )

        # Add country boundaries and legend to axes
        north_korea_boundary_clipped.boundary.plot(
            edgecolor="#e41a1c",
            linewidth=0.5,
            ax=axis,
            alpha=1,
            label="North Korea",
        )
        south_korea_boundary.boundary.plot(
            edgecolor="#1b7837",
            linewidth=0.5,
            ax=axis,
            alpha=1,
            label="South Korea",
        )
        axis.legend(
            shadow=True, edgecolor="white", fontsize=10, loc="lower right"
        )

    # Add caption
    fig.text(
        0.5,
        0.11,
        f"Data Source: European Space Agency",
        ha="center",
        fontsize=16,
    )

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-monthly-means-jul-2018-jun-2020.png",
    )
)

### Magnitude Change

#### Georeferenced

In [ ]:
# Extract metadata from GeoTiff
metadata = stl.extract_geotiff_metadata(monthly_means[0])
metadata

In [ ]:
# Export monthly NO2 change (magnitude)
export_files = [
    (
        magnitude_change_jul_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-jul-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_aug_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-aug-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_sep_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-sep-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_oct_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-oct-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_nov_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-nov-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_dec_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-dec-2018-2019.tif",
        ),
    ),
    (
        magnitude_change_jan_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-jan-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_feb_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-feb-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_mar_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-mar-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_apr_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-apr-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_may_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-may-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_jun_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-jun-2019-2020.tif",
        ),
    ),
    (
        magnitude_change_jul_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-magnitude-jul-2019-2020.tif",
        ),
    ),
]

for (array, outpath) in export_files:
    rd.export_array(array, outpath, metadata)

#### Non-Georeferenced

In [ ]:
# Plot and export July 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["07"],
    post_change=no2_dict["2019"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "July 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "July 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-jul-2018-2019.png",
    )
)

In [ ]:
# Plot and export August 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["08"],
    post_change=no2_dict["2019"]["08"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "August 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "August 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-aug-2018-2019.png",
    )
)

In [ ]:
# Plot and export September 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["09"],
    post_change=no2_dict["2019"]["09"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "September 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "September 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-sep-2018-2019.png",
    )
)

In [ ]:
# Plot and export October 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["10"],
    post_change=no2_dict["2019"]["10"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "October 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "October 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-oct-2018-2019.png",
    )
)

In [ ]:
# Plot and export November 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["11"],
    post_change=no2_dict["2019"]["11"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "November 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "November 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-nov-2018-2019.png",
    )
)

In [ ]:
# Plot and export December 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["12"],
    post_change=no2_dict["2019"]["12"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "December 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "December 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-dec-2018-2019.png",
    )
)

In [ ]:
# Plot and export January 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["01"],
    post_change=no2_dict["2020"]["01"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "January 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "January 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-jan-2019-2020.png",
    )
)

In [ ]:
# Plot and export February 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["02"],
    post_change=no2_dict["2020"]["02"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "February 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "February 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-feb-2019-2020.png",
    )
)

In [ ]:
# Plot and export March 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["03"],
    post_change=no2_dict["2020"]["03"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "March 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "March 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-mar-2019-2020.png",
    )
)

In [ ]:
# Plot and export April 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["04"],
    post_change=no2_dict["2020"]["04"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "April 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "April 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-apr-2019-2020.png",
    )
)

In [ ]:
# Plot and export May 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["05"],
    post_change=no2_dict["2020"]["05"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "May 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "May 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-may-2019-2020.png",
    )
)

In [ ]:
# Plot and export June 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["06"],
    post_change=no2_dict["2020"]["06"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "June 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "June 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-jun-2019-2020.png",
    )
)

In [ ]:
# Plot and export July 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["07"],
    post_change=no2_dict["2020"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="magnitude",
    location="South Korea",
    titles=[
        "Jul 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Jul 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-magnitude-jul-2019-2020.png",
    )
)

### Percent Change

#### Georeferenced

In [ ]:
# Extract metadata from GeoTiff
metadata = stl.extract_geotiff_metadata(monthly_means[0])
metadata

In [ ]:
# Export monthly NO2 change (percent)
export_files = [
    (
        percent_change_jul_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-jul-2018-2019.tif",
        ),
    ),
    (
        percent_change_aug_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-aug-2018-2019.tif",
        ),
    ),
    (
        percent_change_sep_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-sep-2018-2019.tif",
        ),
    ),
    (
        percent_change_oct_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-oct-2018-2019.tif",
        ),
    ),
    (
        percent_change_nov_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-nov-2018-2019.tif",
        ),
    ),
    (
        percent_change_dec_2018_2019,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-dec-2018-2019.tif",
        ),
    ),
    (
        percent_change_jan_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-jan-2019-2020.tif",
        ),
    ),
    (
        percent_change_feb_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-feb-2019-2020.tif",
        ),
    ),
    (
        percent_change_mar_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-mar-2019-2020.tif",
        ),
    ),
    (
        percent_change_apr_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-apr-2019-2020.tif",
        ),
    ),
    (
        percent_change_may_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-may-2019-2020.tif",
        ),
    ),
    (
        percent_change_jun_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-jun-2019-2020.tif",
        ),
    ),
    (
        percent_change_jul_2019_2020,
        os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "change",
            "south-korea-no2-mean-change-percent-jul-2019-2020.tif",
        ),
    ),
]

for (array, outpath) in export_files:
    rd.export_array(array, outpath, metadata)

#### Non-Georeferenced

In [ ]:
# Plot and export July 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["07"],
    post_change=no2_dict["2019"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "July 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "July 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-jul-2018-2019.png",
    )
)

In [ ]:
# Plot and export August 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["08"],
    post_change=no2_dict["2019"]["08"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "August 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "August 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-aug-2018-2019.png",
    )
)

In [ ]:
# Plot and export September 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["09"],
    post_change=no2_dict["2019"]["09"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "September 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "September 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-sep-2018-2019.png",
    )
)

In [ ]:
# Plot and export October 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["10"],
    post_change=no2_dict["2019"]["10"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "October 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "October 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-oct-2018-2019.png",
    )
)

In [ ]:
# Plot and export November 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["11"],
    post_change=no2_dict["2019"]["11"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "November 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "November 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-nov-2018-2019.png",
    )
)

In [ ]:
# Plot and export December 2018-2019 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2018"]["12"],
    post_change=no2_dict["2019"]["12"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "December 2018 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "December 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-dec-2018-2019.png",
    )
)

In [ ]:
# Plot and export January 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["01"],
    post_change=no2_dict["2020"]["01"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "January 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "January 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)


rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-jan-2019-2020.png",
    )
)

In [ ]:
# Plot and export February 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["02"],
    post_change=no2_dict["2020"]["02"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "February 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "February 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-feb-2019-2020.png",
    )
)

In [ ]:
# Plot and export March 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["03"],
    post_change=no2_dict["2020"]["03"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "March 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "March 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-mar-2019-2020.png",
    )
)

In [ ]:
# Plot and export April 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["04"],
    post_change=no2_dict["2020"]["04"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "April 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "April 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-apr-2019-2020.png",
    )
)

In [ ]:
# Plot and export May 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["05"],
    post_change=no2_dict["2020"]["05"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "May 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "May 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-may-2019-2020.png",
    )
)

In [ ]:
# Plot and export June 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["06"],
    post_change=no2_dict["2020"]["06"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "June 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "June 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-jun-2019-2020.png",
    )
)

In [ ]:
# Plot and export July 2019-2020 change
fig, ax = stl.plot_monthly_comparison(
    pre_change=no2_dict["2019"]["07"],
    post_change=no2_dict["2020"]["07"],
    extent_file=monthly_means[0],
    country_boundaries=[north_korea_boundary_clipped, south_korea_boundary],
    country_names=["North Korea", "South Korea"],
    change_type="percent",
    location="South Korea",
    titles=[
        "Jul 2019 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Jul 2020 Mean NO2 ($\mathrm{mol \cdot m^{-2}}$)",
        "Change in Mean NO2 (%)",
    ],
    data_source="European Space Agency",
)

rd.save_figure(
    os.path.join(
        "04-graphics-outputs",
        "spatial-plots",
        "south-korea",
        "south-korea-no2-mean-change-percent-jul-2019-2020.png",
    )
)

In [ ]:
# Unload Notebook formatter
%unload_ext nb_black